In [1]:
# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# ▶ Google drive mount or 폴더 클릭 후 구글드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# ▶ 경로 설정 (※ Colab을 활성화시켰다면 보통 Colab Notebooks 폴더가 자동 생성)
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks'

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data 모음/market_basket.csv',header=None)

# 연관규칙 분석을 통한 규매 규칙 탐색

## 문제상황 정의

A마트는 매출이 나날이 줄고 있다. 이의 문제가 진열에 있다고 판단하고 물품의 진열을 다시 기획하고 있다. 잘팔리는 상품끼리 재배치 하려한다.

고객의 구매 데이터를 이용해 이를 조정하려한다

In [3]:
# data 살펴보기
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 프로젝트 기대효과
마트 매출 증가 , 회복

## 프로세스
연관규칙분석을 통한 물품 재배치

apriori 알고리즘을 사용한 규칙 탐색

Support/Confidence/Lift를 활용한 유의미한 규칙 탐색


In [4]:
# data전처리
# 7500,20개의 구매 데이터가 있음
df.shape

(7501, 20)

In [5]:
# 상당히 많은 null값을 관찰할수 있다.
df.isnull().sum()

0        0
1     1754
2     3112
3     4156
4     4972
5     5637
6     6132
7     6520
8     6847
9     7106
10    7245
11    7347
12    7414
13    7454
14    7476
15    7493
16    7497
17    7497
18    7498
19    7500
dtype: int64

In [6]:
records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])
# df에서 해당 values값이 nan값이면 True를 아니면 해당 문자열을 추가하는 코드를 실행 Nan값을 걸러내기 위해 실행

## TransactionEncoder

연관규칙 학습에서 주로 사용되는 기능으로 물체에 대해 개별적인 columns를 만들고 그 아이템이 속하면 1 아니면 0으로 표현됩니다.

In [7]:
from mlxtend.preprocessing import TransactionEncoder
# 라이브러리 불러오기
# te를 encoder로 선언후 데이터를 통과시키기
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)

te_df = pd.DataFrame(te_ary, columns=te.columns_)

In [8]:
te_df.head()
#해석하자면 0번째 컬럼사람은 True에해당하는 물품만 사간거임


,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


## 구매 규칙 탐색

> apriori를 사용한 규칙 탐색

- 연관규칙이란?

 -- 비지도학습, 대규모 거래 데이터로 부터 함께 구매될 규칙을 도축

 -- 특정상품 구매시 이와 연관성이 높은 상품을 추천하는 것(추천 시스템)

 -- 특정 사건이 발생했을때 함께 빈번하게 발생하는 사건의 규칙

- Apriori알고리즘

```
  - 빈발항목집합을 통해 규칙들을 생성하는 알고리즘
  - 빈발항목집합 → 최소지지도 이상을 갖는 항목집합
```

- 장/단점

```
  - 장점 : 많은 연관규칙들을 발견할 수 있음
  - 단점 : 비지니스 측면에서 중요한 의미있는 규칙들을 발견하기 어려움 -> 상당히 쓸데없는 인사이트 까지 발견됨
```

- 사용예시

```
  - 암 데이터에서 빈번이 발생하는 DNA 패턴 탐색
  - 마트에서 장바구니 분석을 통해 상품 추천 및 상품 진열
  - 설비 고장이 발생하기 전 알람코드들의 패턴 탐색
```

- 작동원리

```
 1단계 : 전체 Data set에서 빈번하게 발생하는 사건의 유형을 발견
 2단계 : 최소지지도(Support) 이상을 만족하는 빈발항목 집합을 발견
 3단계 : 빈발항목으로 집합 생성
 4단계 : 2~3단계 반복 수행, 새로운 빈발항목집합이 생기지 않을 때까지
 5단계 : 빈발항목 집합을 활용하여 연관규칙 생성
 ```

In [9]:
te_df.isnull().sum()

 asparagus           0
almonds              0
antioxydant juice    0
asparagus            0
avocado              0
                    ..
whole wheat pasta    0
whole wheat rice     0
yams                 0
yogurt cake          0
zucchini             0
Length: 120, dtype: int64

In [10]:
# 최대 3개까지의 리스트가 들어간 연관규칙 탐색

from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemset = apriori(te_df,
                           min_support = 0.005,# 최소5% 이상으로 발생할 확률
                           max_len=3,
                           use_colnames=True# 이건 컬럼이름을 사용할지 ->true로 설정하면 인지하기 편함
)
frequent_itemset['length'] = frequent_itemset['itemsets'].apply(lambda x :len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)

In [11]:
# 전체연관규칙 출력
frequent_itemset

,support,itemsets,length
60,0.238368,(mineral water),1
27,0.179709,(eggs),1
83,0.174110,(spaghetti),1
33,0.170911,(french fries),1
20,0.163845,(chocolate),1
...,...,...,...
646,0.005066,"(eggs, tomatoes, mineral water)",3
648,0.005066,"(spaghetti, eggs, olive oil)",3
674,0.005066,"(soup, frozen vegetables, mineral water)",3
680,0.005066,"(grated cheese, ground beef, mineral water)",3


In [12]:
# ▶ 2개 이상의 Item 품목들이 있는 연관 규칙 출력
frequent_itemset[frequent_itemset['length'] >= 2].head(10)

,support,itemsets,length
503,0.059725,"(spaghetti, mineral water)",2
227,0.052660,"(chocolate, mineral water)",2
292,0.050927,"(eggs, mineral water)",2
478,0.047994,"(milk, mineral water)",2
432,0.040928,"(ground beef, mineral water)",2
236,0.039195,"(spaghetti, chocolate)",2
439,0.039195,"(spaghetti, ground beef)",2
299,0.036528,"(spaghetti, eggs)",2
277,0.036395,"(eggs, french fries)",2
387,0.035729,"(frozen vegetables, mineral water)",2


In [13]:
# 특정 규칙을 찾고싶을때 정확한 규칠을 넣어줘야한다.
frequent_itemset[frequent_itemset['itemsets'] == frozenset(('bacon','asparagus'))]
# bacon과 asparagus가 같이 있는 경우를 출력해봄 그외에도 판단하에 관련이 있어보이는 물품을을 엮어 검색할수 있음
# 본인의 주장을 입증가능

,support,itemsets,length


In [14]:
# 상위에서 생성한 Rule에 대해, 추가 지표들에 대한 탐색
# metric과 treshold 설정 필요
association_rules_df = association_rules(frequent_itemset,
                                         metric = 'confidence',
                                         min_threshold=0.005)
association_rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
...,...,...,...,...,...,...,...,...,...
1935,"(spaghetti, pancakes)",(olive oil),0.025197,0.065858,0.005066,0.201058,3.052910,0.003407,1.169224
1936,"(olive oil, pancakes)",(spaghetti),0.010799,0.174110,0.005066,0.469136,2.694478,0.003186,1.555746
1937,(spaghetti),"(olive oil, pancakes)",0.174110,0.010799,0.005066,0.029096,2.694478,0.003186,1.018846
1938,(olive oil),"(spaghetti, pancakes)",0.065858,0.025197,0.005066,0.076923,3.052910,0.003407,1.056037


**antecedents** : 연관규칙의 선행자(antecedent)를 나타내는 열입니다. 선행자는 연관분석에서 어떤 사건이 일어나기 전에 발생하는 사건이나 조건을 의미합니다. 일반적으로 선행자는 조건절이라고도 부릅니다.

**consequents** : 연관규칙의 후행자(consequent)를 나타내는 열입니다. 후행자는 연관분석에서 선행자가 발생한 후에 일어나는 사건이나 결과를 의미합니다. 후행자는 결과절이라고도 불립니다.

**antecedent support** : 선행자의 지지도를 나타내는 열입니다. 지지도는 연관분석에서 특정 아이템이 전체 데이터에서 얼마나 자주 나타나는지를 측정하는 값으로, 해당 아이템의 발생 빈도를 전체 데이터의 크기로 나눈 비율입니다.

**consequent support** : 후행자의 지지도를 나타내는 열입니다. 마찬가지로 후행자의 발생 빈도를 전체 데이터의 크기로 나눈 비율을 나타냅니다.

### ·Support/Confidence/Lift 개념이해
---

* Support(지지도) : 전체 거래 항목 중 해당 규칙이 나올 확률
> "얼마나 자주 나오는 규칙이냐?"

* Confidence(신뢰도) : 조건부확률이라고도 하며, A의 거래중에서 B가 포함된 거래의 확률
> "기저귀 살때, 맥주를 같이 살 확률이 높은가?"

* Lift(향상도) : 임의로 B가 나올 확률(자연확률) 대비 A와 B가 같이 나올 확률의 비
> "자연스럽게 구매하는 확률보다 높은가?"

> 전체 b가 나올확률 대비 a가 나오고 b가 나올 확률

Lift가 1보다 크면 선행자와 후행자 간에 양의 상관관계

In [15]:
# 특정규칙을 찾고 싶을 때
# consequents(후행자)에서 eggs인것만 골라서 추출
# support의 경우 첫번째 컬럼을 예로들면 전체 거래중에 물을사고 계란을 같이 사는 경우가 5%를 뜻함
association_rules_df[association_rules_df['consequents']==frozenset({'eggs'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158
14,(spaghetti),(eggs),0.174110,0.179709,0.036528,0.209801,1.167446,0.005239,1.038081
17,(french fries),(eggs),0.170911,0.179709,0.036395,0.212949,1.184961,0.005681,1.042232
29,(chocolate),(eggs),0.163845,0.179709,0.033196,0.202604,1.127397,0.003751,1.028711
35,(milk),(eggs),0.129583,0.179709,0.030796,0.237654,1.322437,0.007509,1.076009
...,...,...,...,...,...,...,...,...,...
1812,"(french fries, frozen vegetables)",(eggs),0.019064,0.179709,0.005199,0.272727,1.517602,0.001773,1.127900
1821,"(spaghetti, turkey)",(eggs),0.016531,0.179709,0.005199,0.314516,1.750138,0.002229,1.196659
1853,(tomato juice),(eggs),0.030396,0.179709,0.005066,0.166667,0.927423,-0.000396,0.984349
1912,"(tomatoes, mineral water)",(eggs),0.024397,0.179709,0.005066,0.207650,1.155478,0.000682,1.035263


In [16]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemset = apriori(te_df,
                           min_support = 0.005,
                           max_len=5,
                           use_colnames=True)

frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)

In [17]:
frequent_itemset

,support,itemsets,length
60,0.238368,(mineral water),1
27,0.179709,(eggs),1
83,0.174110,(spaghetti),1
33,0.170911,(french fries),1
20,0.163845,(chocolate),1
...,...,...,...
646,0.005066,"(eggs, tomatoes, mineral water)",3
648,0.005066,"(spaghetti, eggs, olive oil)",3
674,0.005066,"(soup, frozen vegetables, mineral water)",3
680,0.005066,"(grated cheese, ground beef, mineral water)",3


In [18]:
#  lift가 1이상인 유의미한 규칙만 탐색 (랜덤 적인 효과 대비 우수) (보통 1이상만 사용 그이하는 중요하다고 판단 X)
rules = association_rules(frequent_itemset, metric="lift", min_threshold=1).sort_values(by = ['lift',  'confidence','support'], ascending =False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1303,(pasta),(escalope),0.015731,0.079323,0.005866,0.372881,4.700812,0.004618,1.468107
1302,(escalope),(pasta),0.079323,0.015731,0.005866,0.073950,4.700812,0.004618,1.062867
1786,(pasta),(shrimp),0.015731,0.071457,0.005066,0.322034,4.506672,0.003942,1.369601
1787,(shrimp),(pasta),0.071457,0.015731,0.005066,0.070896,4.506672,0.003942,1.059374
662,(whole wheat pasta),(olive oil),0.029463,0.065858,0.007999,0.271493,4.122410,0.006059,1.282270
...,...,...,...,...,...,...,...,...,...
1504,(escalope),(tomatoes),0.079323,0.068391,0.005466,0.068908,1.007555,0.000041,1.000555
723,(escalope),(frozen vegetables),0.079323,0.095321,0.007599,0.095798,1.005011,0.000038,1.000528
722,(frozen vegetables),(escalope),0.095321,0.079323,0.007599,0.079720,1.005011,0.000038,1.000432
1194,"(frozen vegetables, mineral water)",(french fries),0.035729,0.170911,0.006133,0.171642,1.004279,0.000026,1.000883


In [19]:
# 보통 통계난 확률의 평균은 잘 안보긴하는데 참고용으로 한번 구해봄
rules['support'].mean(), rules['confidence'].mean()

(0.008956302187364828, 0.15777570774772962)

In [20]:
# 많이 발견되는 규칙(Support참고)
rulse = association_rules(frequent_itemset,metric='support',min_threshold = 0.05).sort_values(by = ['support', 'lift', 'confidence'],ascending = False)
rulse
# 스파게티 산사람이 미네랄워터를 사는 경우가 6%로 생각보다 높게 나옴

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
5,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158


In [27]:
# 무조건 같이 사는 규칙(Confidence)(A를 살때 B를 같이 살 확률)
rulse =association_rules(frequent_itemset,metric='confidence', min_threshold = 0.05).sort_values(by = ['confidence','support','lift'], ascending=False)
rulse
#높을수록 a를 샀을때 b를 같이 살 확률이 높은거임 lift와 같이 봐야 좋다고 볼수있음
# confidence가 높을수록 전후의 강한 상관관계가 있을 확률이 높음


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1530,"(frozen vegetables, soup)",(mineral water),0.007999,0.238368,0.005066,0.633333,2.656954,0.003159,2.077178
1470,"(olive oil, soup)",(mineral water),0.008932,0.238368,0.005199,0.582090,2.441976,0.003070,1.822476
915,"(frozen vegetables, olive oil)",(mineral water),0.011332,0.238368,0.006532,0.576471,2.418404,0.003831,1.798297
569,"(soup, milk)",(mineral water),0.015198,0.238368,0.008532,0.561404,2.355194,0.004909,1.736520
1226,"(soup, chocolate)",(mineral water),0.010132,0.238368,0.005599,0.552632,2.318395,0.003184,1.702471
...,...,...,...,...,...,...,...,...,...
519,(eggs),"(frozen vegetables, mineral water)",0.179709,0.035729,0.009065,0.050445,1.411898,0.002645,1.015498
527,(eggs),(soup),0.179709,0.050527,0.009065,0.050445,0.998387,-0.000015,0.999914
530,(eggs),(grated cheese),0.179709,0.052393,0.009065,0.050445,0.962821,-0.000350,0.997949
948,(milk),"(tomatoes, mineral water)",0.129583,0.024397,0.006532,0.050412,2.066321,0.003371,1.027396
